In [ ]:

import geopandas as gpd
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import networkx as nx
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import rasterio
from scipy.spatial import cKDTree
import seaborn as sns
import xarray as xr

print(os.getcwd())

from fragility_curves import *
from helper.one import (
    read_csv,
    geocode_location,
    process_locations_csv
)
from helper.two import (
    render_tif_image,
    get_tif_value
)
from helper.tree_height_assignment import (
    load_raster_data,
    assign_tree_heights_to_nodes
)

INPUT_FILE = '/root/projs/gcpb/data/GCPB_red_names.csv'
OUTPUT_FILE = 'data/output_base.csv'

df = read_csv(INPUT_FILE)

if df is not None:
    # print(df.head())
    print(f"Successfully read the CSV file: {INPUT_FILE}")
else:
    print("Failed to read the CSV file. Please check the file path and the read_csv function.")
print(df.head())


In [ ]:
# make nominatim geolocator
geo = Nominatim(user_agent="geoapiExercises")
# process_locations_csv(input_file=INPUT_FILE, output_file=OUTPUT_FILE)

In [ ]:
geocoded_Data = pd.read_csv(OUTPUT_FILE)
geocoded_Data.head()

In [ ]:
# write a function to plot the data using plotly 

def plot_data(data):
    # Make a copy of the dataframe to avoid modifying the original
    plot_df = data.copy()
    
    # Convert Latitude and Longitude to numeric, errors='coerce' will convert failures to NaN
    plot_df['Latitude'] = pd.to_numeric(plot_df['Latitude'], errors='coerce')
    plot_df['Longitude'] = pd.to_numeric(plot_df['Longitude'], errors='coerce')
    
    # Drop rows where conversion failed
    plot_df = plot_df.dropna(subset=['Latitude', 'Longitude'])
    
    # Use scatter_map instead of the deprecated scatter_mapbox
    fig = px.scatter_map(plot_df, lat="Latitude", lon="Longitude", hover_data=["Location", "Label"],
                         color="Label", zoom=3, height=600)
    fig.update_layout(mapbox_style="open-street-map")
    fig.show()
plot_data(geocoded_Data)

In [4]:

def plot_with_connections(data, connections=None):
    """
    Plot the data points on a map with lines connecting specific points.
    
    Args:
        data (pd.DataFrame): DataFrame with Location, Latitude, Longitude columns
        connections (list): List of tuples with format [(from_index, to_index), ...] 
                           indicating which points to connect
    """
    # Make a copy of the dataframe to avoid modifying the original
    plot_df = data.copy()
    
    # Convert Latitude and Longitude to numeric, errors='coerce' will convert failures to NaN
    plot_df['Latitude'] = pd.to_numeric(plot_df['Latitude'], errors='coerce')
    plot_df['Longitude'] = pd.to_numeric(plot_df['Longitude'], errors='coerce')
    
    # Drop rows where conversion failed
    plot_df = plot_df.dropna(subset=['Latitude', 'Longitude'])
    
    # Create a figure with scatter points
    fig = px.scatter_map(plot_df, 
                         lat="Latitude", 
                         lon="Longitude", 
                         hover_data=["Location", "Label"],
                         color="Label", 
                         zoom=3, 
                         height=600)
    
    # If connections are provided, add lines between specified points
    if connections:
        for from_idx, to_idx in connections:
            # Check if indices are valid
            if from_idx < len(plot_df) and to_idx < len(plot_df):
                # Get coordinates
                from_lat = plot_df.iloc[from_idx]['Latitude']
                from_lon = plot_df.iloc[from_idx]['Longitude']
                to_lat = plot_df.iloc[to_idx]['Latitude']
                to_lon = plot_df.iloc[to_idx]['Longitude']
                
                # Check if coordinates are valid numbers
                if (pd.notna(from_lat) and pd.notna(from_lon) and 
                    pd.notna(to_lat) and pd.notna(to_lon)):
                    
                    # Add a line trace
                    fig.add_trace(go.Scattermap(
                        mode = "lines",
                        lon = [from_lon, to_lon],
                        lat = [from_lat, to_lat],
                        line = {"width": 2, "color": "red"},
                        showlegend = False
                    ))
    
    fig.update_layout(mapbox_style="open-street-map")
    fig.show()



# Function to connect locations by name instead of index
def connect_locations_by_name(data, connections_by_name):
    """
    Connect locations by their names rather than indices.
    
    Args:
        data (pd.DataFrame): DataFrame with Location, Latitude, Longitude columns
        connections_by_name (list): List of tuples with format [(from_name, to_name), ...]
    """
    # Create connections list with indices
    connections = []
    
    for from_name, to_name in connections_by_name:
        # Find indices for the location names
        from_idx = data[data['Location'] == from_name].index.tolist()
        to_idx = data[data['Location'] == to_name].index.tolist()
        
        # Only add connection if both locations were found
        if from_idx and to_idx:
            connections.append((from_idx[0], to_idx[0]))
    
    # Plot with the index-based connections
    plot_with_connections(data, connections)

In [ ]:
# plot_with_connections(geocoded_Data, connections_example)

# Elvanfoot,80,
# Windyhill,81
# Denny,82,
# Longannet,83
# Inverkip,84
# Hunterston,85
# Nielston,86
# Strathaven,87
# Torness,88
# Eccles,89
# Beauly,91
# Blackhillock,92,,
# Peterhead,93,,
# Kintore,94,,
# Tealing,95,,
# Errochty,96,,
# Rothienorman,97,,
# Harker,9,,
# Stella West,10,,
# Penwortham,11,,
# Deeside,12,,
# Daines,13,,
# Thorpe Marsh,14,,
# Drax,15,,
# Keadby,16,,
# Ratcliffe,17,,
# Feckenham,18,,
# Walpole,19,,
# Bramford,20,,
# Pelham,21,,
# East Claydon,22,,
# Melksham,23,,
# Bramley,24,,
# London,25,,
# Kemsley,26,,
# Sellinge,27,,
# Lovedean,28,,
# Exeter,29,,


# links 
# 91 - 92 (Beauly - Blackhillock)
# 92 - 97 (Blackhillock - Rothienorman)
# 97 - 93 (Rothienorman - Peterhead)
# 97 - 94 (Rothienorman - Kintore)
# 94 - 95 (Kintore - Tealing)
# 95 - 96 (Tealing - Errochty)
# 96 - 91 (Errochty - Beauly)
# 95 - 83 (Tealing - Longannet)
# 83 - 82 (Longannet - Denny)
# 96 - 82 (Errochty - Denny)
# 96 - 81 (Errochty - Windyhill)
# 81 - 80 (Windyhill - Elvanfoot)
# 80 - 84 (Elvanfoot - Inverkip)
# 84 - 85 (Inverkip - Hunterston)
# 85 - 86 (Hunterston - Nielston)
# 86 - 87 (Nielston - Strathaven)
# 81 - 86 (Windyhill - Nielston)
# 82 - 87 (Denny - Strathaven)
# 83 - 87 (Longannet - Strathaven)
# 87 - 88 (Strathaven - Torness)
# 88 - 89 (Torness - Eccles)
# 83 - 89 (Longannet - Eccles)
# 87 - 80 (Strathaven - Elvanfoot)
# 80 - 9 (Elvanfoot - Harker)
# 89 - 10 (Eccles - Stella West)
# 9 - 10 (Harker - Stella West)
# 9 - 11 (Harker - Penwortham)
# 11 - 15 (Penwortham - Drax)
# 10 - 15 (Stella West - Drax)
# 11 - 13 (Penwortham - Daines)
# 13 - 12 (Daines - Deeside)
# 13 - 15 (Daines - Drax)
# 13 - 14 (Daines - Thorpe Marsh)
# 12 - 18 (Deeside - Feckenham)
# 13 - 18 (Daines - Feckenham)
# 14 - 15 (Thorpe Marsh - Drax)
# 14 - 16 (Thorpe Marsh - Keadby)
# 15 - 16 (Drax - Keadby)
# 16 - 19 (Keadby - Walpole)
# 16 - 17 (Keadby - Ratcliffe)
# 16 - 22 (Keadby - East Claydon)
# 16 - 21 (Keadby - Pelham)
# 17 - 18 (Ratcliffe - Melksham)
# 17 - 22 (Ratcliffe - East Claydon)
# 19 - 20 (Walpole - Bramford)
# 18 - 23 (Feckenham - Melksham)
# 23 - 29 (Melksham - Exeter)
# 29 - 28 (Exeter - Lovedean)
# 23 - 24 (Melksham - Thorpe Marsh)
# 24 - 28 (Bramley - Lovedean)
# 24 - 25 (Bramley - London)
# 22 - 21 (East Claydon - Pelham)
# 22 - 23 (East Claydon - Melksham)
# 22 - 25 (East Claydon - London)
# 20 - 21 (Bramford - Pelham)
# 20 - 26 (Bramford - Kemsley)
# 21 - 25 (Pelham - London)
# 25 - 26 (London - Kemsley)
# 26 - 27 (Kemsley - Sellinge)
# 27 - 28 (Sellinge - Lovedean)


connections_by_name_links = [
    ('Beauly','Blackhillock'), 
    ('Blackhillock','Rothienorman'),
    ('Rothienorman','Peterhead'),
    ('Rothienorman','Kintore'),
    ('Kintore','Tealing'),
    ('Tealing','Errochty'),
    ('Errochty','Beauly'),
    ('Tealing','Longannet'),
    ('Longannet','Denny'),
    ('Errochty','Denny'),
    ('Errochty','Windyhill'),
    ('Windyhill','Elvanfoot'),
    ('Elvanfoot','Inverkip'),
    ('Inverkip','Hunterston'),
    ('Hunterston','Nielston'),
    ('Nielston','Strathaven'),
    ('Windyhill','Nielston'),
    ('Denny','Strathaven'),
    ('Longannet','Strathaven'),
    ('Strathaven','Torness'),
    ('Torness','Eccles'),
    ('Longannet','Eccles'),
    ('Strathaven','Elvanfoot'),
    ('Elvanfoot','Harker'),
    ('Eccles','Stella West'),
    ('Harker','Stella West'),
    ('Harker','Penwortham'), #
    ('Penwortham','Drax'),
    ('Stella West','Drax'),
    ('Penwortham','Daines'),
    ('Daines','Deeside'),
    ('Daines','Drax'),
    ('Daines','Thorpe Marsh'),
    ('Deeside', 'Feckenham'),
    ('Daines','Feckenham'),
    ('Thorpe Marsh','Drax'),
    ('Thorpe Marsh','Keadby'),
    ('Drax','Keadby'),
    ('Keadby','Walpole'),
    ('Keadby','Ratcliffe'),
    ('Keadby','East Claydon'),
    ('Keadby','Pelham'),
    ('Ratcliffe','Melksham'),
    ('Ratcliffe','East Claydon'),
    ('Walpole','Bramford'),
    ('Feckenham','Melksham'),
    ('Melksham','Exeter'),
    ('Exeter','Lovedean'),
    ('Melksham','Thorpe Marsh'),
    ('Bramley','Lovedean'),
    ('Bramley','London'),
    ('East Claydon','Pelham'),
    ('East Claydon','Melksham'),
    ('East Claydon','London'),
    ('Bramford','Pelham'),
    ('Bramford','Kemsley'),
    ('Pelham','London'),
    ('London','Kemsley'),
    ('Kemsley','Sellinge'),
    ('Sellinge','Lovedean'),
    ]
connect_locations_by_name(geocoded_Data, connections_by_name_links)

In [ ]:
# make graph object with networkx

reduced_network_graph = nx.Graph()

# use nodes and connections above 
nodes = geocoded_Data[['Location', 'Latitude', 'Longitude']]
connections = connections_by_name_links

# add nodes to graph
for index, row in nodes.iterrows():
    reduced_network_graph.add_node(row['Location'], pos=(row['Longitude'], row['Latitude']))

# add edges to graph
for from_name, to_name in connections:
    reduced_network_graph.add_edge(from_name, to_name)

# draw graph
nx.draw(reduced_network_graph, with_labels=True)
plt.show()


Fragility Curves

In [ ]:
# Example fragility parameters (should be estimated from data)
example_tree_params = (-17.28, 0.42, 0.13)  # Example for tree failure
example_pds_params = (-12.18, 0.071, 0.05)  # Example for PDS component failure

# Example usage
height = 14  # meters
wind_speed = 60  # m/s
wind_angle = 45  # degrees

print("Tree Failure Probability:", compute_tree_failure_probability(height, wind_speed, example_tree_params))
print("PDS Component Failure Probability:", compute_pds_component_failure(wind_speed, wind_angle, example_pds_params))
print("Tree Fall Load:", compute_tree_fall_load(height))


In [ ]:
render_tif_image('data/FTY_2018_010m_03035_V1_0.tif')

In [ ]:
# Get value at specific coordinates
longitude = -1.257677
latitude = 51.752022
value = get_tif_value('data/FTY_2018_010m_03035_V1_0.tif', longitude, latitude)
print(f"Value at ({longitude}, {latitude}): {value}")

In [ ]:
# read in tif file in data folder
crs = 3035


with rasterio.open('data/FTY_2018_010m_03035_V1_0.tif') as src:
    wind_speed_data = src.read(1)
    # wind_speed_transform = src.transform
    # wind_speed_crs = src.crs


In [ ]:
# # show tif file
# plt.imshow(wind_speed_data)
# plt.show()
from PIL import Image
im = Image.open('data/FTY_2018_010m_03035_V1_0.tif')
im.show()

In [ ]:
import psutil

print(f"Available memory: {psutil.virtual_memory().available / (1024 ** 3):.2f} GB")


In [ ]:
import rasterio

tif_path = "data/FTY_2018_010m_03035_V1_0.tif"

try:
    with rasterio.open(tif_path) as src:
        print(f"File opened successfully: {src.meta}")
except Exception as e:
    print(f"Error opening file: {e}")


In [ ]:
with rasterio.open(tif_path) as src:
    window = rasterio.windows.Window(0, 0, 500, 500)  # Adjust size as needed
    small_region = src.read(1, window=window)
    plt.imshow(small_region, cmap='viridis')
    plt.show()


In [ ]:
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show

def render_tif_image(tif_path, downsample_factor=10):
    """
    Reads a .tif file and renders a downsampled version to avoid memory issues.
    """
    with rasterio.open(tif_path) as src:
        # Read with downsampling (reduces memory use)
        data = src.read(1, out_shape=(src.height // downsample_factor, src.width // downsample_factor))
        
        plt.figure(figsize=(10, 6))
        plt.imshow(data, cmap='viridis')
        plt.colorbar(label="Forest Type")
        plt.title("Copernicus Forest Type 2018 (UK)")
        plt.show()


render_tif_image('data/FTY_2018_010m_03035_V1_0.tif')

In [ ]:
from rasterio.warp import transform
from pyproj import Transformer

def get_tif_value(tif_path, lon, lat):
    """
    Given a longitude and latitude, return the corresponding raster value from a .tif file.
    """
    with rasterio.open(tif_path) as src:
        # Transform from WGS84 (lat/lon) to raster's CRS
        transformer = Transformer.from_crs("EPSG:4326", src.crs.to_string(), always_xy=True)
        x, y = transformer.transform(lon, lat)
        
        # Convert world coordinates to raster row/column
        row, col = src.index(x, y)
        
        # Read only the single pixel value instead of full dataset
        value = src.read(1, window=((row, row + 1), (col, col + 1)))[0, 0]

        return value


# Get value at specific coordinates
longitude = -4.872699
latitude = 55.906034
value = get_tif_value('data/FTY_2018_010m_03035_V1_0.tif', longitude, latitude)
print(f"Value at ({longitude}, {latitude}): {value}")



In [ ]:
from shapely.geometry import LineString
import numpy as np

def get_line_coordinates(coord1, coord2, num_points=10):
    """
    Generate intermediate coordinates between two points using linear interpolation.
    
    Args:
        coord1 (tuple): (lon1, lat1) of the first node
        coord2 (tuple): (lon2, lat2) of the second node
        num_points (int): Number of intermediate points
    
    Returns:
        list: List of (lon, lat) coordinates along the transmission line
    """
    line = LineString([coord1, coord2])
    distances = np.linspace(0, line.length, num_points)
    
    return [line.interpolate(distance).coords[0] for distance in distances]

# Example usage:
node_A = (-1.5, 52.5)  # Example substation 1 (lon, lat)
node_B = (-0.5, 53.0)  # Example substation 2 (lon, lat)

line_coords = get_line_coordinates(node_A, node_B, num_points=5)
print(line_coords)  # List of interpolated coordinates


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import LineString
import numpy as np

# Example Power Grid Graph
G = nx.Graph()

# Add Nodes (ID, {Longitude, Latitude})
G.add_node("A", pos=(-1.5, 52.5))  # Example substation 1
G.add_node("B", pos=(-0.5, 53.0))  # Example substation 2
G.add_node("C", pos=(-1.0, 52.8))  # Example substation 3

# Add Edges (Transmission Lines Between Nodes)
G.add_edge("A", "B")
G.add_edge("A", "C")
G.add_edge("B", "C")

def interpolate_line_coords(coord1, coord2, num_points=10):
    """
    Generate intermediate coordinates between two nodes using linear interpolation.
    """
    line = LineString([coord1, coord2])
    distances = np.linspace(0, line.length, num_points)
    return [line.interpolate(distance).coords[0] for distance in distances]

# Extract Transmission Line Coordinates
transmission_lines = []
for edge in G.edges:
    node1, node2 = edge
    coord1, coord2 = G.nodes[node1]['pos'], G.nodes[node2]['pos']
    line_coords = interpolate_line_coords(coord1, coord2, num_points=5)
    transmission_lines.append(line_coords)

# Visualizing the Power Grid
plt.figure(figsize=(8, 6))

# Plot Nodes
pos = nx.get_node_attributes(G, 'pos')
nx.draw(G, pos, with_labels=True, node_color='red', edge_color='gray', node_size=300)

# Plot Transmission Lines (Interpolated Points)
for line in transmission_lines:
    lons, lats = zip(*line)
    plt.plot(lons, lats, 'b--', alpha=0.6)  # Dashed blue line for transmission lines

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Reduced UK Power Grid Model")
plt.grid(True)
plt.show()


In [ ]:
# do as above for the power grid 'reduced_network_graph'

reduced_network_graph 


In [ ]:
# Example usage
raster_path = "tree_height_data.tif"  # Example raster file
power_grid = nx.Graph()  # Example electricity model
power_grid.add_nodes_from(
    [(0, {"coordinates": (0.5, 51.5)}), (1, {"coordinates": (1.0, 52.0)})]
)

data, transform, no_data = load_raster_data(raster_path)
power_grid = assign_tree_heights_to_nodes(power_grid, data, transform, no_data)

for node, attrs in power_grid.nodes(data=True):
    print(f"Node {node}: Tree Height = {attrs['tree_height']}")

In [21]:


def display_image_from_nc(nc_path, var_name, time_index=0, cmap='viridis'):
    """
    Display an image from a NetCDF (.nc) file for the specified variable at a specific time index.
    
    Args:
        nc_path (str): Path to the NetCDF file.
        var_name (str): The variable name in the NetCDF file to display.
        time_index (int): The index of the time dimension to plot (default is 0).
        cmap (str): Colormap for the plot.
    """
    # Open the NetCDF file
    ds = xr.open_dataset(nc_path)
    
    # Get the variable data (assumes it has dimensions [time, lat, lon])
    var_data = ds[var_name]
    
    # If it's a 3D array, slice it by time
    if var_data.ndim == 3:
        var_data = var_data.isel(time=time_index)
    
    # Plot the variable data as an image
    plt.figure(figsize=(10, 6))
    var_data.plot.imshow(cmap=cmap)
    plt.title(f"{var_name} from {nc_path} at time index {time_index}")
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()


def get_value_from_nc(nc_path, var_name, lon, lat):
    """
    Given a longitude and latitude, return the value for that location from a NetCDF (.nc) file.

    Args:
        nc_path (str): Path to the NetCDF file.
        var_name (str): The variable name in the NetCDF file.
        lon (float): Longitude of the point.
        lat (float): Latitude of the point.

    Returns:
        float: The value at the specified location.
    """
    # Open the NetCDF file
    ds = xr.open_dataset(nc_path)

    # Get the variable data (assumes it has dimensions [lat, lon])
    var_data = ds[var_name]

    # Extract latitudes and longitudes
    lons = ds['longitude'].values
    lats = ds['latitude'].values

    # Use KDTree to find the nearest grid point (efficient search)
    tree = cKDTree(np.vstack((lons, lats)).T)
    dist, idx = tree.query([lon, lat])

    # Get the value for the closest point
    value = var_data.values[idx]
    
    return value

In [ ]:
ds = xr.open_dataset('data/sfcWind_hadukgrid_uk_1km_mon_202001-202012.nc')
print(ds)


In [ ]:
# see timessteps 
print(ds['time'])

In [ ]:
display_image_from_nc('data/sfcWind_hadukgrid_uk_1km_mon_202001-202012.nc', var_name='sfcWind', time_index=0)